* Author: HZQ, ZTR
* Last modified: 2019/1/23

# Main Parts

## Import required packages

In [0]:
def name():
    pass

In [ ]:
!pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
import xgboost
import math

## prepared process


### read data online (colab)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null 
!apt-get -y install -qq google-drive-ocamlfuse fuse 
from google.colab import auth 
auth.authenticate_user() 
from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default() 
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p driver
!google-drive-ocamlfuse driver
import os
os.chdir("driver/loyalty_prediction_data")
!ls

In [ ]:
# load data
def load_data(filename, is_train=True):
    data = pd.read_csv(filename, index_col=0)
    card_id = data['card_id'].values
    drop_list = ['first_active_month', 'card_id']
    if is_train:
        drop_list.append('target')
        y = data['target'].values
    data = data.drop(drop_list, axis=1)
    # fill missing values
    for c in data.columns:
        data[c] = data[c].fillna(data[c].dropna().median())
    x = data.values
    if is_train:
        return x, y, card_id
    else:
        return x, card_id
    
x_train, y_train, _ = load_data('train.csv')
x_test, card_id_test = load_data('test.csv', is_train=False)

### read data offline

## model training

### XGBoost model

In [0]:
# model & training
model = xgboost.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100)
model.fit(x_train, y_train)

In [ ]:
pred_train = model.predict(x_train)
loss_train = math.sqrt(np.sum((pred_train - y_train) ** 2) / len(y_train))
print('Training RMSE loss: {}'.format(loss_train))

In [ ]:
# predicting
pred_test = model.predict(x_test)
df_test = pd.DataFrame({'card_id': card_id_test, 'target': pred_test})
df_test.to_csv('submission_xgboost.csv', index=False)

### baseline model (e.g Linear regression)

## model validation


# Undertesting Parts